In [1]:
import tensorflow as tf
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

tf.python.control_flow_ops = tf


Using TensorFlow backend.


In [2]:
from src.data import Data
from src.plothelp import *

In [3]:
data = Data("img")
data.load_pickle("100x100.pickle ")
data.resize_img(size=[64,64])
X_train, X_test, y_train, y_test = data.train_test_split()

In [4]:
X_train = np.asarray(X_train)
X_test = np.asanyarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X_train = np.swapaxes(X_train,1,3)
X_test = np.swapaxes(X_test,1,3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4698, 3, 64, 64) (522, 3, 64, 64) (4698,) (522,)


In [5]:
def oneHot(y_train, y_test):
    from sklearn import preprocessing
    
    le = preprocessing.LabelEncoder()
    le.fit(y_train)
    numeric_labels =le.transform(y_train)
    numeric_test_labels = le.transform(y_test)
    n_classes =4
    y_one_hot = np.eye(4)[numeric_labels]
    y_test_code = np.eye(4)[numeric_test_labels]
    
    return y_one_hot, y_test_code
    
    

In [6]:
y_oh, y_test_oh= oneHot(y_train, y_test)

In [8]:
# Create the model
model = Sequential()
model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(3,64,64)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(3,64,64)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
# Compile model
lrate = 0.01
nb_epoch=20
decay = lrate/nb_epoch
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 64, 62, 62)    1792        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 64, 31, 31)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 64, 29, 29)    36928       maxpooling2d_3[0][0]             
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 64, 14, 14)    0           convolution2d_4[0][0]            
___________________________________________________________________________________________

In [9]:
# Fit the model
model.fit(X_train, y_oh, validation_data=(X_test, y_test_oh), nb_epoch=20, batch_size=100, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test_oh, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 4698 samples, validate on 522 samples
Epoch 1/20
4698/4698 [==============================] - 180s - loss: 12.1998 - acc: 0.2420 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 2/20
4698/4698 [==============================] - 181s - loss: 12.2653 - acc: 0.2390 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 3/20
4698/4698 [==============================] - 183s - loss: 12.2555 - acc: 0.2395 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 4/20
4698/4698 [==============================] - 185s - loss: 12.2687 - acc: 0.2388 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 5/20
4698/4698 [==============================] - 183s - loss: 12.2687 - acc: 0.2388 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 6/20
4698/4698 [==============================] - 189s - loss: 12.2653 - acc: 0.2390 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 7/20
4698/4698 [==============================] - 185s - loss: 12.2653 - acc: 0.2390 - val_loss: 12.2584 - val_acc: 0.2395
Epoch 8/20
4698/4698 [============================